# Web Scraping - Listagem de todos os jogos cadastrados no BGG

Projeto para fazer a respagem dos dados de todos os jogos cadastrados no BGG, e avaliar quais são os anos que a maior partes deles foi lançada.

- Dados: https://boardgamegeek.com/browse/boardgame

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

No site do BGG há 1330 páginas com jogos, então primeiramente temos que fazer a coleta de todos as páginas do site

In [41]:
pages = []

for i in range(1, 1331):
    url = 'https://boardgamegeek.com/browse/boardgame/page/' + str(i)
    pages.append(url)

In [ ]:
for url in pages:
    page = requests.get(url).text 
    soupBGG = BeautifulSoup (page, 'html.parser')
    
    for tab in soupBGG.find_all("tr", attrs={"id": "row_"}):
               
        classe = tab.find(class_="collection_thumbnail")
        if classe.find('img') is not None:
            img = classe.find('img')['src']
            print (img)
        else:
            print('sem imagem')


                 
               

Em seguida temos que fazer o requeste de todos as urls coletadas 


In [123]:
lista_jogos = []

for url in pages:
    page = requests.get(url).text 
    soupBGG = BeautifulSoup (page, 'html.parser')
    
    for tab in soupBGG.find_all("tr", attrs={"id": "row_"}):
        
        dados_jogo = []

        # filtrando o ranking
        ranking = tab.find(class_="collection_rank")
        rank = ranking.text.strip() # .strip() para tirar os espaços antes e depois da string  
        dados_jogo.append(rank)


        # filtrando a capa      
        classe = tab.find(class_="collection_thumbnail")
        if classe.find('img') is not None:
            capa = classe.find('img')['src']
            dados_jogo.append(capa)
        else:
            dados_jogo.append('Missing image')          
                          

                
        # filtrando o link da pag do jogo
        link_jogo = tab.find(class_="primary")
        link_jogo["href"]
        link_pag = "https://boardgamegeek.com" + link_jogo["href"]
        dados_jogo.append(link_pag)

        # filtrando o nome do jogo
        nome_dos_jogos = tab.find(class_="primary")
        nome = nome_dos_jogos.text
        dados_jogo.append(nome)

        # filtrando a data de lançamento
        ano_lancamento = tab.find(class_="smallerfont dull")
        if ano_lancamento != None:
            ano = ano_lancamento.text.strip('(').strip(')') # strip é para tirar os parenteses
            dados_jogo.append(ano)
        else:
            ano = "Not Available"
            dados_jogo.append(ano)
            
            
        # filtrando a classificação do jogo
        notas = tab.find(class_="collection_bggrating") # usando somente find, porque quero a primeira nota
        nota_GeekRating = notas.text.strip() # .strip() para tirar os espaços antes e depois da string
        dados_jogo.append(nota_GeekRating)

        # add as partes para a lista "lista_jogos"
        lista_jogos.append(dados_jogo) 

Agora podemos tranformar essa lista em um dataFrame e trabalhar nos dados com o pandas

In [134]:
jogos_df = pd.DataFrame(lista_jogos, columns=['BG_Rank', 'Box_Image', 'Link', 'Title', 'Year', 'Geek_Rating'])

jogos_df.head()

,BG_Rank,Box_Image,Link,Title,Year,Geek_Rating
0,1,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg,https://boardgamegeek.com/boardgame/174430/gloomhaven,Gloomhaven,2017,8.511
1,2,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png,https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1,Pandemic Legacy: Season 1,2015,8.444
2,3,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/img/4Od3GYCiqptga0VbmyumPbJlBsU=/fit-in/64x64/filters:strip_icc()/pic3490053.jpg,https://boardgamegeek.com/boardgame/224517/brass-birmingham,Brass: Birmingham,2018,8.417
3,4,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZQ4rxw__micro/img/LUkXZhd1TO80eCiXMD3-KfnzA6k=/fit-in/64x64/filters:strip_icc()/pic3536616.jpg,https://boardgamegeek.com/boardgame/167791/terraforming-mars,Terraforming Mars,2016,8.273
4,5,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE-FgFA__micro/img/2gymaKs35_2yj7eyyA6cYyVmd9c=/fit-in/64x64/filters:strip_icc()/pic3727516.jpg,https://boardgamegeek.com/boardgame/233078/twilight-imperium-fourth-edition,Twilight Imperium: Fourth Edition,2017,8.260


In [125]:
jogos_df.tail()

,BG_Rank,Box_Image,Link,Title,Year,Geek_Rating
132969,N/A,https://cf.geekdo-images.com/zxVVmggfpHJpmnJY9j-k1w__micro/img/6OZE0C5RkxED46qU_A2oi1t7208=/fit-in/64x64/filters:strip_icc()/pic1657689.jpg,https://boardgamegeek.com/boardgame/353889/circles,Circles,2022,N/A
132970,N/A,https://cf.geekdo-images.com/WW7KDwiA7g3CgnVxA3cSrg__micro/img/52eCTVn-kJF3ZMFrwmkn5N__h3k=/fit-in/64x64/filters:strip_icc()/pic6595331.png,https://boardgamegeek.com/boardgameexpansion/353890/witchstone-full-moon,Witchstone: Full Moon,2022,N/A
132971,N/A,https://cf.geekdo-images.com/zxVVmggfpHJpmnJY9j-k1w__micro/img/6OZE0C5RkxED46qU_A2oi1t7208=/fit-in/64x64/filters:strip_icc()/pic1657689.jpg,https://boardgamegeek.com/boardgameexpansion/353891/hexa-ruins-island-crocodiles-and-drakes,ヘクサ・ルインス 〜クロコダイルとドレイクの島〜 (Hexa Ruins: The Island of Crocodiles and Drakes -),2021,N/A
132972,N/A,https://cf.geekdo-images.com/zxVVmggfpHJpmnJY9j-k1w__micro/img/6OZE0C5RkxED46qU_A2oi1t7208=/fit-in/64x64/filters:strip_icc()/pic1657689.jpg,https://boardgamegeek.com/boardgame/353892/5-sekund-dzieci-kontra-dorosli,5 Sekund: Dzieci kontra Dorośli,2021,N/A
132973,N/A,https://cf.geekdo-images.com/zxVVmggfpHJpmnJY9j-k1w__micro/img/6OZE0C5RkxED46qU_A2oi1t7208=/fit-in/64x64/filters:strip_icc()/pic1657689.jpg,https://boardgamegeek.com/boardgame/353906/madness-hour,Madness Hour / マッドネスアワー,2021,N/A


In [126]:
jogos_df.shape 

(132974, 6)

In [127]:
jogos_df.to_csv('jogos_bgg.csv', index=False)

Avaliando o anos que os jogos foram lançados 

In [129]:
jogos_df.Year.unique()

array(['2017', '2015', '2018', '2016', '2020', '2012', '2005', '2011',
       '2013', '2007', '2019', '2014', '2002', '2004', '2008', '2006',
       '1876', '2010', '1995', '2009', '1997', '1982', '1993', '1999',
       '-2200', '1991', '2000', '2003', '1986', '2021', '1992', '1998',
       '1996', '1964', '1979', '1980', '1985', '1994', '1475', '2001',
       '1990', '1989', '1983', '1630', '1959', '1977', '1800', '1925',
       '1984', '1850', '1810', '1988', 'Not Available', '1987', '1971',
       '-3000', '1978', '1587', '1981', '1973', '1974', '762', '1962',
       '1848', '1903', '1938', '1947', '1960', '1948', '2022', '1895',
       '1930', '1972', '1906', '1976', '1967', '1745', '1864', '1946',
       '1970', '400', '1883', '1965', '1975', '1966', '1425', '1701',
       '1600', '1939', '1969', '1909', '1942', '1904', '700', '1932',
       '1963', '1968', '1780', '1870', '1663', '1921', '1715', '1951',
       '1956', '550', '1955', '1885', '1860', '1830', '1887', '1796',
       

Ao verificar os anos de lançamentos, nota-se varios números negativos e um valor 2<br>
Verificando o motivo desses valores, contata-se que o site catalogou todos os jogos já mendionados em algum momento da hitória e endo assim colocou a data de quando o jogo foi mencionado pela primeirra vez na história, como o jogo Liubo, datado de ~ 700AC.

Já o valor 2 (A Song of Ice & Fire: Tabletop Miniatures Game – Targaryen card update pack), verificou-se que foi um valor digitado errado, e na verade é 2021.Nesse caso esse dado deve ser arrumado


In [136]:
jogos_df.query(" Year == '2' ")

,BG_Rank,Box_Image,Link,Title,Year,Geek_Rating
130634,N/A,https://cf.geekdo-images.com/zxVVmggfpHJpmnJY9j-k1w__micro/img/6OZE0C5RkxED46qU_A2oi1t7208=/fit-in/64x64/filters:strip_icc()/pic1657689.jpg,https://boardgamegeek.com/boardgameexpansion/348016/song-ice-fire-tabletop-miniatures-game-targaryen-c,A Song of Ice & Fire: Tabletop Miniatures Game – Targaryen card update pack,2,N/A


In [137]:
jogos_df.loc[jogos_df.Year == "2", "Year"] = "2021"

In [139]:
jogos_df.query(" Title == 'A Song of Ice & Fire: Tabletop Miniatures Game – Targaryen card update pack' ")

,BG_Rank,Box_Image,Link,Title,Year,Geek_Rating
130634,N/A,https://cf.geekdo-images.com/zxVVmggfpHJpmnJY9j-k1w__micro/img/6OZE0C5RkxED46qU_A2oi1t7208=/fit-in/64x64/filters:strip_icc()/pic1657689.jpg,https://boardgamegeek.com/boardgameexpansion/348016/song-ice-fire-tabletop-miniatures-game-targaryen-c,A Song of Ice & Fire: Tabletop Miniatures Game – Targaryen card update pack,2021,N/A


In [140]:
jogos_df.to_csv('jogos_bgg.csv')